# Cross-validated logistic regression with threshold optimization on fertility dataset
## Dataset: Fertility (https://archive.ics.uci.edu/ml/datasets/Fertility)

## Data Transformation

### Package and Data Set-Up 

In [43]:
import pandas as pd
import numpy as np
import sklearn
data = pd.read_csv(r'C:\Users\vince\Desktop\ESCP\Courses\Machine Learning (Python)\Group Task 1\Fertility_Diagnosis.csv')
data.head()

,Season,Age,Disease,Accident,Surgery,Fever,Alcohol,Smoking,Sitting,Diagnosis
0,-0.33,0.69,0,1,1,0,0.8,0,0.88,N
1,-0.33,0.94,1,0,1,0,0.8,1,0.31,O
2,-0.33,0.50,1,0,0,0,1.0,-1,0.50,N
3,-0.33,0.75,0,1,1,0,1.0,-1,0.38,N
4,-0.33,0.67,1,1,0,0,0.8,-1,0.50,O


### Categorical Variables as Dummies

In [44]:
data = pd.get_dummies(data, columns = ['Season'], prefix = ["Season"])
data = data.rename( columns=
                   {'Season_-1.0': 'Winter', 'Season_-0.33': 'Spring', 'Season_0.33':'Summer' , 'Season_1.0':'Fall'})
data = pd.get_dummies(data, columns = ['Disease','Accident','Surgery','Fever','Alcohol','Smoking','Diagnosis'], 
                      prefix = ["Disease",'Accident','Surgery','Fever', 'Alcohol','Smoking','Diagnosis'])
data.head()

,Age,Sitting,Winter,Spring,Summer,Fall,Disease_0,Disease_1,Accident_0,Accident_1,...,Alcohol_0.2,Alcohol_0.4,Alcohol_0.6,Alcohol_0.8,Alcohol_1.0,Smoking_-1,Smoking_0,Smoking_1,Diagnosis_N,Diagnosis_O
0,0.69,0.88,0,1,0,0,1,0,0,1,...,0,0,0,1,0,0,1,0,1,0
1,0.94,0.31,0,1,0,0,0,1,1,0,...,0,0,0,1,0,0,0,1,0,1
2,0.50,0.50,0,1,0,0,0,1,1,0,...,0,0,0,0,1,1,0,0,1,0
3,0.75,0.38,0,1,0,0,1,0,0,1,...,0,0,0,0,1,1,0,0,1,0
4,0.67,0.50,0,1,0,0,0,1,0,1,...,0,0,0,1,0,1,0,0,0,1


### One Hot Encoding (dropping 1 dummy column per category)

In [45]:
data = data.drop(columns=['Winter','Disease_0','Accident_0','Surgery_0','Fever_-1','Alcohol_0.2','Smoking_-1','Diagnosis_N'])
data.head()

,Age,Sitting,Spring,Summer,Fall,Disease_1,Accident_1,Surgery_1,Fever_0,Fever_1,Alcohol_0.4,Alcohol_0.6,Alcohol_0.8,Alcohol_1.0,Smoking_0,Smoking_1,Diagnosis_O
0,0.69,0.88,1,0,0,0,1,1,1,0,0,0,1,0,1,0,0
1,0.94,0.31,1,0,0,1,0,1,1,0,0,0,1,0,0,1,1
2,0.50,0.50,1,0,0,1,0,0,1,0,0,0,0,1,0,0,0
3,0.75,0.38,1,0,0,0,1,1,1,0,0,0,0,1,0,0,0
4,0.67,0.50,1,0,0,1,1,0,1,0,0,0,1,0,0,0,1


### Splitting Features and Label

In [46]:
x = data.drop(columns = ['Diagnosis_O'])
y = data['Diagnosis_O']
x.head()

,Age,Sitting,Spring,Summer,Fall,Disease_1,Accident_1,Surgery_1,Fever_0,Fever_1,Alcohol_0.4,Alcohol_0.6,Alcohol_0.8,Alcohol_1.0,Smoking_0,Smoking_1
0,0.69,0.88,1,0,0,0,1,1,1,0,0,0,1,0,1,0
1,0.94,0.31,1,0,0,1,0,1,1,0,0,0,1,0,0,1
2,0.50,0.50,1,0,0,1,0,0,1,0,0,0,0,1,0,0
3,0.75,0.38,1,0,0,0,1,1,1,0,0,0,0,1,0,0
4,0.67,0.50,1,0,0,1,1,0,1,0,0,0,1,0,0,0


## Considering Business Context

In this scenario, a fertility clinic needs to identify which individuals are potentially infertile based on their risk factors to determine priority for fertility testing. Testing is resource and time intensive, and therefore the clinic wants to prioritize testing individuals that have a high probability of being infertile based on their input values. 

This model is developed from the perspective of the clinic, and with this tool, the clinic will be able to rapidly input a set of common risk factors from patients (alcohol consumption, smoking status, past illnesses and surgeries) to determine who to test for infertility. 

## Types of Prediction Errors for Model

The two types of prediction errors that the classification model could make are false positive and false negative errors. A false positive in this scenario signifies that the model predicted a male individual to be infertile when in actuality, they are fertile. The individual would then need to be tested, which would validate the false positive result. As the testing is time and resource intensive, the associated cost would revolve around lost time and effort. A rough estimate of this cost is \\$120 (labor & resource cost).

A false negative error is more serious in this case. A patient is indeed infertile, but the model predicts that he is fertile. The clinic could be at risk for legal troubles, as well as providing misinformation to their patients and damaging their trust. While it is difficult to estimate the financial impact this could have on the clinic, let's assume that there is a 0.1 percent chance of a lawsuit costing \\$500,000. The associated cost would then be $500 per false negative error.

the FP-FN cost ratio in this case would be \\$120/\$500, or 0.24.

## Defining the Cost Function

In [47]:
def cost_function(y_pred, y_test):
    TP = sum(y_pred[(y_test == 1)] == 1)
    TN = sum(y_pred[(y_test == 0)] == 0)
    FP = sum(y_pred[(y_test == 0)] == 1)
    FN = sum(y_pred[(y_test == 1)] == 0)
    
    cost = FP*120 + FN*500
    return cost

Splitting Data for 10-Fold Cross Validation

## 10-Fold CV Classification Model
#### Note: Due to the small size of the dataset, StratifiedKFold function was used to stratify the target variable

In [51]:
skf = sklearn.model_selection.StratifiedKFold(n_splits=10, shuffle=True, random_state=100)
log = sklearn.linear_model.LogisticRegression()
out = []
for train_index, test_index in skf.split(x, y):
    cost = []
    x_train, x_test = x.iloc[train_index, :], x.iloc[test_index, :]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    log.fit(x_train, y_train)
    y_prob = log.predict_proba(x_test)
    y_prob = y_prob[:, 1]
    
    for k in range(0,100):
        y_pred = y_prob >= k/100
        cost.append(cost_function(y_pred, y_test))
    out.append(cost)
    
print(out[0:1])

[[1080, 1080, 1080, 1080, 960, 960, 960, 960, 720, 480, 480, 480, 480, 480, 480, 360, 360, 360, 360, 860, 740, 740, 740, 740, 620, 620, 620, 620, 620, 620, 620, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500]]


## Best Threshold and Associated Cost
#### including dataframe preparation from output

In [52]:
ind = np.arange(0,1,1/100)
df = pd.DataFrame(out)
df = df.transpose()
df = df.set_index(ind, drop=True)
df['threshold_mean'] = df.mean(axis=1)
print(df.head())
print("\n")
print("Best threshold (min): " + str(df['threshold_mean'].idxmin()), "\n" + "Average cost function: "+ str(df['threshold_mean'].min()))

         0     1     2     3     4     5     6     7    8    9  threshold_mean
0.00  1080  1080  1080  1080  1080  1080  1080  1080  960  960          1056.0
0.01  1080  1080  1080  1080  1080  1080  1080  1080  960  960          1056.0
0.02  1080  1080   960  1080  1080   960  1080  1080  960  960          1032.0
0.03  1080  1460   960   960  1080   960   960   960  960  960          1034.0
0.04   960  1340   960   960  1080   960  1460   960  840  960          1048.0


Best threshold (min): 0.43 
Average cost function: 600.0
